Finde ein Model welches den Trainingsdatensatz perfekt lernt (Accuracy = 1.). Im Notebook kannst du dir für das Model auch die Testgenauigkeit ausgeben lassen.

In dieser Aufgabe siehst du wie man Overfitting erkennen kann: das Model ist auf die Trainingsdaten überangepasst und generalisiert
nicht auf anderen Daten. Der Konstante Klassifizierer wiederum ist ein Beispiel für Underfitting: das Model ist sowohl auf den Trainingsdaten als auch auf neuen Daten gleich schlecht. Die Schwierigkeit besteht darin ein Modell zu finden das weder underfittet noch overfittet. In der Challenge ist es deine Aufgabe ein Model zu finden, dass möglichst gut generalisiert. (Die Challenge werden wir in den nächsten Tagen freischalten.)

In [ ]:
import torch
import numpy as np
import torch.nn as nn

D = np.load('data/train_data.npy')

In [ ]:
def train(D):

    train, test = D[:200], D[200:]  # Die Auswertung der Aufgabe basiert auf diesem Split

    # Hyper-parameter
    n_steps = ...
    learning_rate = ...
    input_size = 13
    output_size = 1

    # Trainings-Daten vorbereiten
    X = train[:, :-1].astype(np.float32)
    y = train[:, -1].astype(np.float32)
    X_train = torch.from_numpy(X)
    y_train = torch.from_numpy(y)
    feature_means = torch.mean(X_train, dim=0)

    # Test-Daten vorbereiten
    X = test[:, :-1].astype(np.float32)
    y = test[:, -1].astype(np.float32)
    X_test = torch.from_numpy(X)
    y_test = torch.from_numpy(y)

    # Modell definieren
    class MLP(nn.Module):
        def __init__(self, input_size):
            super().__init__()
            self.layers = nn.Sequential(
                ...  # Definiere hier dein eigenes Model
            )

        def forward(self, x):
            x = x - feature_means
            out = self.layers(x)
            return out

    model = MLP(input_size)

    # loss and optimizer
    # checkout: https://pytorch.org/docs/stable/nn.html#torch.nn.BCEWithLogitsLoss
    criterion = nn.BCEWithLogitsLoss()  # sigmoid + binary cross entropy

    # optimizer
    # Dokumentation: https://pytorch.org/docs/stable/optim.html#torch.optim.Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # Alternative zu Adam
    # Dokumentation: https://pytorch.org/docs/stable/optim.html#torch.optim.SGD
    # momentum = 0.9  # Wert zwischen 0. und 1.
    # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    # trainieren des Modells
    for e in range(n_steps):
        # forward pass
        outputs = model.forward(X_train)[:, 0]  # Xw (linear layer)
        loss = criterion(outputs, y_train)  # sigmoid and cross-entropy loss

        # backward pass (automatically computes gradients)
        optimizer.zero_grad()  # reset gradients (torch accumulates them)
        loss.backward()  # computes gradients

        # Optimierungsschritt durchfuehren
        optimizer.step()

        # berechne Trainings-Accuracy
        outputs = model.forward(X_train)[:, 0]
        pred_y = outputs > 0
        is_correct = torch.eq(pred_y, y_train.byte()).float()
        accuracy_train = torch.mean(is_correct).item()

        # berechne Test-Accuracy
        outputs = model.forward(X_test)[:, 0]
        pred_y = outputs > 0
        is_correct = torch.eq(pred_y, y_test.byte()).float()
        accuracy_test = torch.mean(is_correct).item()

    print(f'Epoch {e}, Loss: {loss:.4f}, Acc train: {accuracy_train:.2f},' \
          f'Acc test: {accuracy_test:.2f}')
    return model
